In [35]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

spb = pd.DataFrame()

def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

os.chdir(r'C:\Users\AmanovRA\Текущее\СПБ_Валере_2й_квартал')
for path, folders, files in os.walk('.'):
    if not folders:
        for file in files:
            if file[-4:] == 'xlsx' and path != '.\Вконтакте':
                source = f'{os.path.basename(path)}, {file[:file.find(".")]}'
                temp_df = pd.read_excel(path + '\\' + file)
                temp_df['Источник'] = [source for i in range(temp_df.shape[0])]
                temp_df = temp_df.loc[:, ['Телефон', 'Источник']]
                temp_df = temp_df.drop(temp_df[temp_df['Телефон'].isnull()].index)
                temp_df['Телефон'] = temp_df['Телефон'].apply(to_string)
                spb = spb.append(temp_df, ignore_index=True)
 
temp_df = pd.read_excel(r'C:\Users\AmanovRA\Текущее\СПБ_Валере_2й_квартал\Вконтакте\Звонки ВК 2 квартал.xlsx')
temp_df['Источник'] = ['Вконтакте' for i in range(temp_df.shape[0])]
temp_df = temp_df.loc[:, ['Телефон', 'Источник']]
temp_df = temp_df.drop(temp_df[temp_df['Телефон'].isnull()].index)
temp_df['Телефон'] = temp_df['Телефон'].apply(to_string)
spb = spb.append(temp_df, ignore_index=True)                
        
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'


mobil = spb['Телефон'].to_list()
        

for i, j in enumerate(mobil):
    if type(j) == str:
        mobil[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
        mobil[i] = mobil[i].lstrip(' ')
        mobil[i] = mobil[i].rstrip(' ')
        if re.search(pattern2, mobil[i]):
            mobil[i] = re.sub(pattern2, '', mobil[i])
        mobil[i] = ''.join([x for x in mobil[i] if x not in '()-'])  
            

for i, j in enumerate(mobil):
    if type(j) == str:
        if len(j) == 7:
            mobil[i] = '7812' + mobil[i]
        elif len(j) == 11 and j[0] == '8':
            mobil[i] = '7' + j[1:]
        elif len(j) == 10:
            mobil[i] = '7' + j
            
            
spb['Телефон'] = mobil   

spb = spb.drop_duplicates(subset='Телефон').reset_index(drop=True)

In [55]:

au_telefons = pd.read_excel('AU_2_quart.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)

au_telefons['Telefons'] = lst_search

# lst_with_coincidenc_telefon = []
# lst_with_coincidenc_source = []
# for index, row in enumerate(lst_search):
#     lst_with_coincidenc_telefon.append([])
#     lst_with_coincidenc_source.append([])
#     for i, j in enumerate(nvs['Телефон']):
#         if str(j) in row:
#             lst_with_coincidenc_telefon[index].append(str(j))
#             lst_with_coincidenc_source[index].append(nvs['Источник'][i])
#             break
            
# au_telefons['NVS-телефон'], au_telefons['NVS-источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source           
            
# def join_list(lst):
#     return ', '.join([str(i) for i in lst])

# au_telefons['NVS-телефон'], au_telefons['NVS-источник'] = au_telefons['NVS-телефон'].apply(join_list), au_telefons['NVS-источник'].apply(join_list)

# au_telefons.to_excel('NVS_Valere.xlsx')

In [56]:
vu_telefons = pd.read_excel('VU_2_quart.xlsx')

phones = vu_telefons['Телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y
            
phones2            


[['79262284434'],
 ['79262284434'],
 ['78129330310', '79217708681'],
 ['78129330310', '79217708681'],
 ['78129330310', '79217708681'],
 ['78129330310', '79217708681'],
 ['78127520764', '79312366260'],
 ['78127520764', '79312366260'],
 ['78127520764', '79312366260'],
 ['78125343980', '78123493490'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['321617161', '78122957227', '78123432008', '79219478641'],
 ['78125502904', '79219322850

In [57]:
def concat(lst):
    arr = []
    for i in lst:
        arr.append(' '.join(i))
    return arr    

In [58]:
phones2 = concat(phones2)

In [59]:
vu_telefons['Telefons'] = phones2

In [60]:
au_telefons.to_excel('au.xlsx')

vu_telefons.to_excel('vu.xlsx')

In [61]:
au_telefons = pd.read_excel('au.xlsx')

In [67]:
au_telefons['Telefons'] = au_telefons['Telefons'].apply(str)

lst_search = au_telefons['Telefons'].to_list()

lst_with_coincidenc_telefon = []
lst_with_coincidenc_source = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append([])
    lst_with_coincidenc_source.append([])
    for i, j in enumerate(spb['Телефон']):
        if str(j) in row:
            lst_with_coincidenc_telefon[index].append(str(j))
            lst_with_coincidenc_source[index].append(spb['Источник'][i])
            break
            
au_telefons['SPB-телефон'], au_telefons['SPB-источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source           
            
def join_list(lst):
    return ', '.join([str(i) for i in lst])

au_telefons['SPB-телефон'], au_telefons['SPB-источник'] = au_telefons['SPB-телефон'].apply(join_list), au_telefons['SPB-источник'].apply(join_list)

au_telefons.to_excel('SPB_Valere.xlsx')

AttributeError: 'list' object has no attribute 'dtypes'